# 06. Groupby와 Pivot table(with타이타닉호 데이터)

In [1]:
from IPython.display import Image
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## apply() - 함수를 적용
* apply()는 데이터 전처리시 굉장히 많이 활용하는 기능입니다.
* 좀 더 복잡한 logic을 컬럼 혹은 DataFrame에 적용하고자 할 때 사용합니다.

In [3]:
# 남자 537, 여자 271, 아이 83명으로 구성된 데이터 테이블
df['who'].value_counts()

man      537
woman    271
child     83
Name: who, dtype: int64

In [4]:
def transform_who(x):
    if x == 'man':
        return '남자'
    elif x == 'woman':
        return '여자'
    else:
        return '아이'

In [5]:
# apply를 통해 함수 적용
df['who'].apply(transform_who)

0      남자
1      여자
2      여자
3      여자
4      남자
       ..
886    남자
887    여자
888    여자
889    남자
890    남자
Name: who, Length: 891, dtype: object

In [6]:
# 분포 확인
df['who'].apply(transform_who).value_counts()

남자    537
여자    271
아이     83
Name: who, dtype: int64

In [7]:
def transform_who(x):
    return x['fare'] / x['age']

In [8]:
df.apply(transform_who, axis=1)

0      0.329545
1      1.875876
2      0.304808
3      1.517143
4      0.230000
         ...   
886    0.481481
887    1.578947
888         NaN
889    1.153846
890    0.242188
Length: 891, dtype: float64

In [9]:
df['survived'].value_counts()

0    549
1    342
Name: survived, dtype: int64

In [10]:
# lambda 함수 사용
df['survived'].apply(lambda x: '생존' if x == 1 else '사망')

0      사망
1      생존
2      생존
3      생존
4      사망
       ..
886    사망
887    생존
888    사망
889    생존
890    사망
Name: survived, Length: 891, dtype: object

In [11]:
# 분포 확인
df['survived'].apply(lambda x: '생존' if x == 1 else '사망').value_counts()

사망    549
생존    342
Name: survived, dtype: int64

## Groupby()
* 데이터를 특정 기준으로 그룹핑할 때 활용합니다. 엑셀의 피봇테이블과 유사합니다.

In [12]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [15]:
df.groupby('sex').mean()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone
sex,,,,,,,,
female,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818,0.000000,0.401274
male,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893,0.930676,0.712305


### 2개 이상의 컬럼으로 그룹

In [16]:
df.groupby(['sex', 'pclass']).mean()

survived        age     sibsp     parch        fare  \
sex    pclass                                                        
female 1       0.968085  34.611765  0.553191  0.457447  106.125798   
       2       0.921053  28.722973  0.486842  0.605263   21.970121   
       3       0.500000  21.750000  0.895833  0.798611   16.118810   
male   1       0.368852  41.281386  0.311475  0.278689   67.226127   
       2       0.157407  30.740707  0.342593  0.222222   19.741782   
       3       0.135447  26.507589  0.498559  0.224784   12.661633   

               adult_male     alone  
sex    pclass                        
female 1         0.000000  0.361702  
       2         0.000000  0.421053  
       3         0.000000  0.416667  
male   1         0.975410  0.614754  
       2         0.916667  0.666667  
       3         0.919308  0.760807

In [17]:
# 1개의 특정 컬럼에 대한 결과 도출
df.groupby(['sex', 'pclass'])['survived'].mean()

sex     pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: survived, dtype: float64

In [20]:
# dataframe 형태로 만들기 - 1
pd.DataFrame(df.groupby(['sex', 'pclass'])['survived'].mean())

survived
sex    pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

In [21]:
# dataframe 형태로 만들기 - 2
df.groupby(['sex', 'pclass'])[['survived']].mean()

survived
sex    pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

### reset_index(): 인덱스 초기화
* 그룹핑된 데이터프레임의 index를 초기화하여 새로운 데이터프레임을 생성합니다.

In [23]:
df.groupby(['sex', 'pclass'])['survived'].mean().reset_index()

sex     pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: survived, dtype: float64

### 다중 컬럼에 대한 결과 도출

In [24]:
# 성별, 좌석등급 별 통계
df.groupby(['sex', 'pclass'])[['survived', 'age']].mean()

survived        age
sex    pclass                     
female 1       0.968085  34.611765
       2       0.921053  28.722973
       3       0.500000  21.750000
male   1       0.368852  41.281386
       2       0.157407  30.740707
       3       0.135447  26.507589

### 다중 통계 함수 적용

In [25]:
# 성별, 좌석등급 별 통계
df.groupby(['sex', 'pclass'])[['survived', 'age']].agg(['mean', 'sum'])

survived            age         
                   mean sum       mean      sum
sex    pclass                                  
female 1       0.968085  91  34.611765  2942.00
       2       0.921053  70  28.722973  2125.50
       3       0.500000  72  21.750000  2218.50
male   1       0.368852  45  41.281386  4169.42
       2       0.157407  17  30.740707  3043.33
       3       0.135447  47  26.507589  6706.42

## Pivot_table()
* 피벗테이블은 엑셀의 피벗과 동작이 유사하며, groupby()와도 동작이 유사합니다.
* 기본 동작 원리는 index, columns, values를 지정하여 피벗합니다.

In [69]:
# 1개 그룹에 대한 단일 컬럼 결과
# index에 그룹을 표기
df.pivot_table(index='who', values='survived')

,survived
who,
child,0.590361
man,0.163873
woman,0.756458


In [70]:
# columns에 그룹을 표기
df.pivot_table(columns='who', values='survived')

who,child,man,woman
survived,0.590361,0.163873,0.756458


In [71]:
# 다중 그룹에 대한 단일 컬럼 결과
df.pivot_table(index=['who', 'pclass'], values='survived')

survived
who   pclass          
child 1       0.833333
      2       1.000000
      3       0.431034
man   1       0.352941
      2       0.080808
      3       0.119122
woman 1       0.978022
      2       0.909091
      3       0.491228

In [72]:
df.pivot_table(index='who', columns='pclass', values='survived')

pclass,1,2,3
who,,,
child,0.833333,1.000000,0.431034
man,0.352941,0.080808,0.119122
woman,0.978022,0.909091,0.491228


In [73]:
# 다중 통계함수 적용
df.pivot_table(index='who', columns='pclass', values='survived', aggfunc=['sum', 'mean'])

sum              mean                    
pclass   1   2   3         1         2         3
who                                             
child    5  19  25  0.833333  1.000000  0.431034
man     42   8  38  0.352941  0.080808  0.119122
woman   89  60  56  0.978022  0.909091  0.491228

# 연습문제

In [26]:
sample = df.copy()
sample.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [27]:
sample['class'].value_counts()

Third     491
First     216
Second    184
Name: class, dtype: int64

## sample 데이터프레임에 대하여 apply()를 활용하여 class 컬럼의 값을 다음과 같이 바꾸고, 분포를 출력후 변경 전과 동일한지 확인하세요

In [28]:
def class_function(x):
    if x == 'Third':
        word = '삼등석'
    elif x == 'First':
        word = '일등석'
    else:
        word = '이등석'
    return word

In [35]:
sample['class'].apply(class_function).value_counts()

삼등석    491
일등석    216
이등석    184
Name: class, dtype: int64

## groupby()를 활용하여 다음을 출력 하세요

### pclass 별 생존율

In [64]:
sample = df.copy()
sample

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [42]:
sample.groupby(['pclass'])['survived'].mean()

pclass
1    0.629630
2    0.472826
3    0.242363
Name: survived, dtype: float64

### embarked 별 생존율 통합 통계

In [43]:
sample.groupby(['embarked'])['survived'].agg(['mean','var'])

,mean,var
embarked,,
C,0.553571,0.248610
Q,0.389610,0.240943
S,0.336957,0.223764


### who, pclass별 생존율, 생존자수

In [46]:
sample.groupby(['who','pclass'])['survived'].agg(['mean','sum'])

mean  sum
who   pclass               
child 1       0.833333    5
      2       1.000000   19
      3       0.431034   25
man   1       0.352941   42
      2       0.080808    8
      3       0.119122   38
woman 1       0.978022   89
      2       0.909091   60
      3       0.491228   56

### 남자의 나이는 남자 나이의 평균으로 채우세요

In [67]:
sample['age'] = sample.groupby(['sex'])['age'].apply(lambda x:x.fillna(x.mean()))

In [68]:
# 결측치 확인
print(sample['age'].isnull().sum())
print(f"age 평균: {sample['age'].mean():.2f}")

0
age 평균: 29.74


## pivot_table()를 활용하여 다음을 출력 하세요

In [74]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### value는 tip에 대한 평균값을 산출합니다.

In [82]:
tips.pivot_table(index='smoker', columns='day',values='tip')

day,Thur,Fri,Sat,Sun
smoker,,,,
Yes,3.030000,2.7140,2.875476,3.516842
No,2.673778,2.8125,3.102889,3.167895


### value는 total_bill에 대한 평균과 합계를 산출합니다.

In [86]:
tips.pivot_table(index='day', columns='time',values='total_bill',aggfunc=['mean','sum'])

mean                 sum         
time      Lunch     Dinner    Lunch   Dinner
day                                         
Thur  17.664754  18.780000  1077.55    18.78
Fri   12.845714  19.663333    89.92   235.96
Sat         NaN  20.441379     0.00  1778.40
Sun         NaN  21.410000     0.00  1627.16